**Tags:** kitchen,recipe,playground

# Welcome Cookbook

Design, create, and deploy a Playground entirely from the Kitchen. Use this notebook to prototype layouts, bind backend routes, and stream changes into the frontend. Each Recipe builds on the previous one—feel free to duplicate sections, branch off into your own Cookbooks, or export snapshots once you are satisfied.

## Recipe 1 · Orientation & Overview

Goals:
- Understand Playground vocabulary (frontend, backend, Kitchen, Recipes, Cookbooks).
- Choose your tenant + Playground namespace.
- Push a simple manifest preview to confirm the round-trip.

📋 **Checklist**
1. Read `docs/PLAYGROUND_GUIDE.md` §Vocabulary and §Interaction Lifecycle.
2. Update the identifiers below to match your tenant and Playground.
3. Run the manifest preview helper to ensure the frontend reflects your change.

In [ ]:
# TODO: customize your namespace identifiers
TENANT = "demo-tenant"
PLAYGROUND = "welcome-control"
REVISION_LABEL = "v0"

print(f"Targeting {TENANT}/{PLAYGROUND} (label={REVISION_LABEL})")

In [ ]:
import json
from pathlib import Path
import os
import sys

LAB_ROOT = Path("..").resolve()
os.environ.setdefault("LAB_ROOT", str(LAB_ROOT))
DATA_PATH = LAB_ROOT / "data" / "interactions.db"
KITCHEN_SRC = LAB_ROOT / "kitchen"
if str(KITCHEN_SRC) not in sys.path:
    sys.path.append(str(KITCHEN_SRC))
if str(LAB_ROOT) not in sys.path:
    sys.path.append(str(LAB_ROOT))

from scripts.manifest import ManifestPublisher, ManifestPublishError
from playground.backend.app.config import get_settings

PLAYGROUND_API_URL = os.getenv("PLAYGROUND_API_URL", "http://localhost:8000/api")
AUTHOR = os.getenv("KITCHEN_AUTHOR", "welcome-cookbook")
publisher = ManifestPublisher(base_url=PLAYGROUND_API_URL)
settings = get_settings()
DATA_STORE_PROVIDER = settings.database_provider
DATA_STORE_LABEL = str(settings.database_path) if settings.database_provider == "sqlite" else settings.database_provider

print(f"Using lab root: {LAB_ROOT}")
print(f"Data store provider: {DATA_STORE_PROVIDER} ({DATA_STORE_LABEL})")
print(f"Manifest API: {PLAYGROUND_API_URL}")
print(f"Publishing as: {AUTHOR}")

## Recipe 2 · Getting Started

Goals:
- Scaffold your Kitchen environment and helper imports.
- Instantiate a control client that talks to the backend.
- Author your first layout tree with at least two sections.

📋 **Checklist**
1. Ensure `pip install -r kitchen/requirements.txt` has been run.
2. Import helper modules (temporary shim below until the layout DSL ships).
3. Draft layout JSON + default widget bindings.

In [ ]:
from pathlib import Path
import json
import os
import sys

LAB_ROOT = Path("..").resolve()
os.environ.setdefault("LAB_ROOT", str(LAB_ROOT))
DATA_PATH = LAB_ROOT / "data" / "interactions.db"
KITCHEN_SRC = LAB_ROOT / "kitchen"
if str(KITCHEN_SRC) not in sys.path:
    sys.path.append(str(KITCHEN_SRC))
if str(LAB_ROOT) not in sys.path:
    sys.path.append(str(LAB_ROOT))

from scripts.manifest import ManifestPublisher, ManifestPublishError
from playground.backend.app.config import get_settings

PLAYGROUND_API_URL = os.getenv("PLAYGROUND_API_URL", "http://localhost:8000/api")
AUTHOR = os.getenv("KITCHEN_AUTHOR", "welcome-cookbook")
publisher = ManifestPublisher(base_url=PLAYGROUND_API_URL)
settings = get_settings()
DATA_STORE_PROVIDER = settings.database_provider
DATA_STORE_LABEL = str(settings.database_path) if settings.database_provider == "sqlite" else settings.database_provider

print(f"Using lab root: {LAB_ROOT}")
print(f"Data store provider: {DATA_STORE_PROVIDER} ({DATA_STORE_LABEL})")
print(f"Manifest API: {PLAYGROUND_API_URL}")
print(f"Publishing as: {AUTHOR}")

In [ ]:
def publish_manifest_snapshot(manifest: dict, *, recipe: str):
    """Send a manifest to the backend and log helpful status messages."""
    try:
        result = publisher.publish(
            tenant=TENANT,
            playground=PLAYGROUND,
            manifest=manifest,
            cookbook="Welcome Cookbook",
            recipe=recipe,
            author=AUTHOR,
            revision_label=REVISION_LABEL,
        )
        checksum_preview = result.checksum[:8]
        print(
            f"✅ Published revision {result.revision} for {recipe} (checksum {checksum_preview}…)."
        )
        return result
    except ManifestPublishError as exc:
        print(
            "⚠️ Publish failed. Ensure the backend is running at",
            PLAYGROUND_API_URL,
            "—",
            exc,
        )
        return None

In [ ]:
preview_result = publish_manifest_snapshot(
    sample_manifest,
    recipe="Orientation & Overview",
)

In [ ]:
layout_tree = {
    "sections": [
        {
            "id": "inputs",
            "title": "Input Controls",
            "widgets": [
                {
                    "type": "prompt",
                    "props": {
                        "label": "Primary Prompt",
                        "placeholder": "Describe your task..."
                    },
                }
            ],
        },
        {
            "id": "outputs",
            "title": "Outputs",
            "widgets": [
                {
                    "type": "markdown",
                    "props": {"value": "Preview results will stream here."},
                }
            ],
        },
    ]
}
print(json.dumps(layout_tree, indent=2))

## Recipe 3 · Tutorial Build

Goals:
- Bind frontend widgets to backend endpoints (prompt relay, telemetry feed).
- Capture and persist manifest revisions.
- Add lightweight validation tests to keep Recipes reproducible.

📋 **Checklist**
1. Define a manifest structure with `version`, `widgets`, and `actions`.
2. Implement a mock backend client that logs operations (replace with real SDK later).
3. Write at least one test that asserts the manifest schema before publishing.

In [ ]:
def build_manifest(layout: dict) -> dict:
    return {
        "version": 1,
        "layout": layout,
        "actions": [
            {
                "id": "submit_prompt",
                "route": "/api/chat",
                "method": "POST",
            }
        ],
    }


manifest_v1 = build_manifest(layout_tree)
publish_result = publish_manifest_snapshot(manifest_v1, recipe="Tutorial Build")

In [ ]:
def test_manifest_has_layout(manifest: dict) -> None:
    assert "layout" in manifest, "Layout missing from manifest"
    assert manifest["layout"].get("sections"), "At least one section required"


test_manifest_has_layout(manifest_v1)
print("Manifest validation passed.")

## Recipe 4 · Sample Playground

Goals:
- Showcase a complete workflow (Ops triage board, support queue, analytics room, etc.).
- Highlight how Kitchen edits, backend routing, and frontend widgets stay in sync.
- Document operational hooks (telemetry capture, health checks, command palette entries).

📋 **Checklist**
1. Describe the scenario in markdown (audience, data sources, control loops).
2. Compose multi-pane layouts with per-pane Recipes.
3. Capture runbooks or LabControl commands required to operate the Playground.

> Example scenario: **Support Triage Playground**
>
> - Pane A: real-time ticket ingest + filters.
> - Pane B: LLM-assisted reply composer tied to `/api/chat`.
> - Pane C: Tail-log stream + health indicators.
> - Backend glue: FastAPI routes for tickets + Cosmos DB partitioned by `tenantId`.
>
> Replace this blurb with your own scenario notes, diagrams, or screenshots.

## Recipe 5 · Advanced Integrations

Goals:
- Connect to remote data (Cosmos DB/Postgres), secrets, and multi-tenant sandboxes.
- Demonstrate resiliency: retries, timeouts, incremental backoff.
- Set up deployment artifacts (Cookbook export, Control Capsule metadata).

📋 **Checklist**
1. Configure credentials via `.env` or secret store (never hardcode).
2. Write adapter functions that read from remote stores and hydrate widgets.
3. Export the Cookbook manifest to `release_artifacts/` for future reuse.

In [ ]:
import os

from playground.backend.app.services.data_store import data_store_context

COSMOS_ENDPOINT = os.getenv("COSMOS_ENDPOINT", "https://localhost:8081")
print(f"Target Cosmos endpoint: {COSMOS_ENDPOINT}")

def fetch_remote_signals(limit: int = 5):
    """Placeholder for Cosmos/Postgres query logic that uses the Playground data store."""
    with data_store_context() as store:
        records = store.list_interactions(limit=limit)
    return [(record.id, record.user_prompt_text) for record in records]

remote_preview = fetch_remote_signals()
for row in remote_preview:
    print(row)

### Export next steps

- [ ] Implement `cookbook.export()` helper (tracked in Kitchen roadmap).
- [ ] Save manifest snapshots under `kitchen/_exports/<tenant>/<playground>/`.
- [ ] Reference this notebook from Control Capsules so new users land here first.